<a href="https://colab.research.google.com/github/shafique5930/web3.0/blob/main/chatbot_without_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%%capture --no-stderr

In [2]:
%pip install langgraph langsmith langchain_google_genai langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.21
    Uninstalling langchain-core-0.3.21:
      Successfully uninstalled langchain-core-0.3.21
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.9
    Uninstalling langchain-0.3.9:
      Successfully uninstalled langchain-0.3.9


In [5]:
import os
from google.colab import userdata
os.environ["LANGCHAIN_PROJECT"] = 'Basic Chatbot'
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
gemini_api_key = userdata.get("GEMINI_API_KEY")

In [7]:
from typing import Annotated
from typing_extensions import TypedDict
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph.message import add_messages
from langgraph.graph.state import StateGraph, START, END

class ChatMessage(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(ChatMessage)

llm = ChatGoogleGenerativeAI(
    model= "gemini-1.5-flash",
    api_key=gemini_api_key,
)
def chatbot(state: ChatMessage):
  return {"messages": [llm.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, 'chatbot')
graph_builder.add_edge('chatbot', END)
graph = graph_builder.compile()
llm.invoke('KIA HALL HAY')

AIMessage(content="That looks like a name, possibly a person's name or a business name.  Without more context, it's difficult to say for sure what it means or refers to.\n", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-d6a2924b-603f-469d-a76e-7d4a82ce862b-0', usage_metadata={'input_tokens': 5, 'output_tokens': 38, 'total_tokens': 43, 'input_token_details': {'cache_read': 0}})

In [13]:
# from IPython.display import Image, display
try: display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
  pass

In [14]:
for event in graph.stream({"messages": [("user", "What do you know about LangGraph?")]}):
    for value in event.values():
        print("Assistant:", value["messages"][-1].content)

Assistant: LangGraph is a relatively new and less widely known project compared to established graph databases like Neo4j or Amazon Neptune.  Information about it is scattered and not comprehensively documented in a central, easily accessible location.  However, based on the limited information available, it appears to be a **graph database specifically designed for natural language processing (NLP) tasks.**

Here's what we can infer:

* **Focus on NLP:** Its core functionality revolves around representing and querying relationships within textual data. This suggests it's optimized for handling the complexities of linguistic structures, unlike general-purpose graph databases that might need more adaptation for NLP applications.

* **Graph Representation of Text:**  LangGraph likely uses graph structures (nodes and edges) to encode the relationships between words, phrases, sentences, or even documents in a text corpus.  This could involve representing semantic relationships, syntactic d

{'Hello:': 'Muhammad Shafique'}


In [11]:
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [("user", user_input)]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)

while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break

        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

User: HI
Assistant: Hi there! How can I help you today?

User: q
Goodbye!
